In [1]:
import pandas as pd
import pyarrow
import matplotlib
import matplotlib.pyplot as plt
#jupyter nbconvert homer_report.ipynb --no-input --to html

# HOMER data integration to WGS of DSD project

## What is HOMER?
HOMER is a software for motif discovery and next-gen sequencing analysis.
we use the findMotifsGenome.pl program to perform de novo motif discovery as well as check the enrichment of known motifs.

## Our process:
We've set up a nextflow pipeline to run HOMER on our hATAC_mATAC regions to find enriched and de-novo motifs. This is an example output data:

In [2]:
homer_output_file = "../25_12/keyTF_motifs_in_peaks.csv"
homer_output = pd.read_csv(homer_output_file)
homer_output.iloc[:, 1:]

,PositionID,Offset,Sequence,Motif Name,Strand,MotifScore
0,15822_Y35,671,CCTTTGTGGG,Sox10(HMG)/SciaticNerve-Sox3-ChIP-Seq(GSE35132...,+,7.741960
1,15822_Y35,685,CCTTTGTGGG,Sox10(HMG)/SciaticNerve-Sox3-ChIP-Seq(GSE35132...,+,7.741960
2,PS.SE42022,1845,CCATTGTCCC,Sox10(HMG)/SciaticNerve-Sox3-ChIP-Seq(GSE35132...,+,8.756874
3,PS.SE42020,1251,CCTTTGTACC,Sox10(HMG)/SciaticNerve-Sox3-ChIP-Seq(GSE35132...,+,7.464307
4,PS.SE42019,239,GAAACAAAGC,Sox10(HMG)/SciaticNerve-Sox3-ChIP-Seq(GSE35132...,-,8.253506
...,...,...,...,...,...,...
942841,83480_Y35_GR.PG44,974,ACATCAAAGG,Tcf3(HMG)/mES-Tcf3-ChIP-Seq(GSE11724)/Homer,+,11.264622
942842,83486_XY1235_PG39,161,GCATCAAAGG,Tcf3(HMG)/mES-Tcf3-ChIP-Seq(GSE11724)/Homer,+,9.213214
942843,83486_XY1235_PG39,582,GCTTCAAAGG,Tcf3(HMG)/mES-Tcf3-ChIP-Seq(GSE11724)/Homer,+,9.170481
942844,GR.PG.PS.SE28,1349,TCATCAAAGG,Tcf3(HMG)/mES-Tcf3-ChIP-Seq(GSE11724)/Homer,+,8.886311


Results columns:
PositionID is the name of the ATAC interval, 
Offset is from the start of the ATAC interval, 
Sequence is the BS, 
Motif Name includes the TF name and the source of the experimental data, 
Strand of the BS,
Score.

The score calculates the likelihood that a given sequence matches the motif represented by the PWM compared to a background model (often assuming random occurrence of bases). Higher log odds scores indicate a better match or stronger similarity between the motif and the input sequence. Homer recommands a threshold around 5-10, and 5 is used as a default. 
*We currently don't use a higher threshold*
This is the distribution of the scores:

In [3]:
summary_stats = homer_output['MotifScore'].describe()
print(summary_stats)

count    942846.000000
mean          7.560098
std           1.316626
min           5.195305
25%           6.634124
50%           7.574695
75%           8.370506
max          12.770999
Name: MotifScore, dtype: float64


As part of the pipeline, we filter the results to keep only data about our TF of interest. 

Out of our list of TF of interest, not all are in HOMER:

In [11]:
tf_list = pd.read_csv("../../useful_files/tf_list.csv", header=None)
tf_list.columns = ['TF']
tf_list['exists_in_homer_data'] = tf_list['TF'].apply(lambda x: any(homer_output['Motif Name'].str.contains(x, case=False)))
tf_list['source'] = tf_list['TF'].apply(lambda x: next((source for source in homer_output['Motif Name'] if x.lower() in source.lower()), None))
tf_list['origin'] = ['mouse', 'human',  None, 'mouse', 'mouse + human', None, 'mouse', 'rat', None, None, None, 'mouse', 'human', 'human', 'mouse', 'human', 'mouse']
tf_list

,TF,exists_in_homer_data,source,origin
0,GATA4,True,Gata4(Zf)/Heart-Gata4-ChIP-Seq(GSE35151)/Homer,mouse
1,LHX9,True,LHX9(Homeobox)/Hct116-LHX9.V5-ChIP-Seq(GSE1168...,human
2,NR5A1,False,None,None
3,AR,True,FoxL2(Forkhead)/Ovary-FoxL2-ChIP-Seq(GSE60858)...,mouse
4,DMRT1,True,DMRT1(DM)/Testis-DMRT1-ChIP-Seq(GSE64892)/Homer,mouse + human
5,SOX8,False,None,None
6,SOX9,True,Sox9(HMG)/Limb-SOX9-ChIP-Seq(GSE73225)/Homer,mouse
7,SOX10,True,Sox10(HMG)/SciaticNerve-Sox3-ChIP-Seq(GSE35132...,rat
8,SRY,False,None,None
9,ESR1,False,None,None


In [8]:
tf_list['source'][7]

'Sox10(HMG)/SciaticNerve-Sox3-ChIP-Seq(GSE35132)/Homer'

So instead of NR5A1 we take NR5A2, but the rest of the 'FALSE' don't have a replacement.

Eventually we save the parsed data to "homer_TFBS_results.prq" from which we extract relevant data according to genomic coordinates using "extractTFBS.R".